In [1]:
%matplotlib inline

In [2]:
import model_utils
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [3]:
data_dir = 'data/masked_images/data/trainval/breeds_cat/'
test_data_dir = 'data/masked_images/data/test/breeds_cat/'
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                        #transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                        #transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
trainloader, valloader, classes = model_utils.load_data('breeds_cat', train_transforms, test_transforms, 1)
#trainloader, valloader, classes, labels = model_utils.load_split_train_val(data_dir, train_transforms, test_transforms, normalize, .25)

In [4]:
use_gpu = torch.cuda.is_available()

model_cat = torchvision.models.resnet50(pretrained=True)
num_ftrs = model_cat.fc.in_features

model_cat.fc = nn.Linear(num_ftrs, len(classes))

if use_gpu:
    model_cat = model_cat.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_cat.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [5]:
model_cat = model_utils.train_model(model=model_cat,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=scheduler,
                           trainloader=trainloader,
                           valloader=valloader,
                           num_epochs=15)

Epoch 0/14
--------------------
train Loss: 2.5730 Acc: 0.0852
val Loss: 2.4725 Acc: 0.1332
--------------------
Epoch 1/14
--------------------
train Loss: 2.5233 Acc: 0.0984
val Loss: 2.3047 Acc: 0.2272
--------------------
Epoch 2/14
--------------------
train Loss: 2.4541 Acc: 0.1298
val Loss: 2.2238 Acc: 0.1749
--------------------
Epoch 3/14
--------------------
train Loss: 2.3582 Acc: 0.1672
val Loss: 1.8926 Acc: 0.4621
--------------------
Epoch 4/14
--------------------
train Loss: 2.1568 Acc: 0.2577
val Loss: 1.6939 Acc: 0.3995
--------------------
Epoch 5/14
--------------------
train Loss: 1.8706 Acc: 0.4472
val Loss: 1.6220 Acc: 0.6005
--------------------
Epoch 6/14
--------------------
train Loss: 1.8219 Acc: 0.5462
val Loss: 1.5952 Acc: 0.5875
--------------------
Epoch 7/14
--------------------
train Loss: 1.7828 Acc: 0.5495
val Loss: 1.5977 Acc: 0.5822
--------------------
Epoch 8/14
--------------------
train Loss: 1.7406 Acc: 0.5705
val Loss: 1.5314 Acc: 0.5849
----

In [8]:
saving_path = './trained_model/resnet50_breed_cat.pth'
torch.save(model_cat, saving_path)

In [9]:
model_utils.print_test_acc(model_cat, test_transforms, normalize, test_data_dir)

Overall accuracy for all breeds 52% 

Accuracy of Abyssinian : 34 %
Accuracy of Bengal : 70 %
Accuracy of Birman : 65 %
Accuracy of Bombay : 92 %
Accuracy of British_Shorthair : 82 %
Accuracy of Egyptian_Mau :  0 %
Accuracy of Maine_Coon : 30 %
Accuracy of Persian : 77 %
Accuracy of Ragdoll : 10 %
Accuracy of Russian_Blue :  0 %
Accuracy of Siamese : 80 %
Accuracy of Sphynx : 92 %
